In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# import data
df = pd.read_csv('../Dog Racing Info/Runners_2015.csv')
df.shape

(703829, 91)

## 'Fin' is our target column

In [3]:
df = df[df['Fin'] != 0]   #0 is not a valid result so I'm remvoing it

### Cleaning up Null values

In [4]:
#Drop columns that are unecessary or have too many nulls
df.drop(columns=['Dog', 'Wwt', 'By1', 'By2', 'Nrm', 'Ntm', 'Dhf', 'Swt', 'Tag', 'Ptk', 'Rem', 'Scr', 'Fby', 'Col',
                 'Ctk', 'Com', 'Ar1', 'Ar2', 'Ar3', 'Ar4'], inplace=True)
#probably find a way to do something with com column too
#probably can find a way to include Ar1-Ar4

In [5]:
df['Fav'].fillna(0, inplace=True)
df['Fav'] = df['Fav'].apply(lambda x: 1 if x == '*' else x)   #make Fav column binary

In [6]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [7]:
df.shape

(489354, 71)

### Cleaning up values and datatypes

In [8]:
df['Trk'] = df['Trk'].replace('`F', 'F')
df = df[(df['Trk']=='F') | (df['Trk']=='M') | (df['Trk']=='S')]   #only keep F, M and S values for Track conditions

In [9]:
fix_grd_values_dict = {'C!':'C', 'E!':'E', 'SC!':'SC', 'scl':'Scl'}
df['Grd'] = df['Grd'].apply(lambda x: fix_grd_values_dict[x] if x in fix_grd_values_dict.keys() else x)   #clean up 

In [10]:
df['Odd'] = df['Odd'].str.replace('\*.', '', regex=True)
df = df[(df['Odd'] != '-----') & (df['Odd'] != '-1.00')]
df['Odd'] = df['Odd'].str.replace('\.$', '', regex=True)
df['Odd'] = df['Odd'].str.replace('\.\.', '.', regex=True)
#remove remaining 6 messed up strings
removed_odds = [x for x in pd.DataFrame(df['Odd'].value_counts()).index if x.count('.') > 1]  #if it has more than 1 decimal its incorrect
df = df[~df['Odd'].isin(removed_odds)]
df['Odd'] = df['Odd'].astype(float)
df['Odd'] = df['Odd'].round(2)

In [11]:
df['Mln'] = df['Mln'].apply(lambda x: x + '-1' if x.count('-') < 1 else x)
half_values_dict = {'5/2-1':'2.5-1', '7/2-1':'3.5-1', '9/2-1':'4.5-1'}
df['Mln'] = df['Mln'].apply(lambda x: half_values_dict[x] if x.count('/') == 1 else x)
df['Mln'] = df['Mln'].apply(lambda x: float(x.split('-')[0]) / float(x.split('-')[1]) + 1)

In [12]:
convert_dst_values_dict = {'5-16':'503', '3-16':'301', '3-8':'603', 'DC':'0', '7-16':'703', 'YARD':'0'}
df['Dst'] = df['Dst'].apply(lambda x: convert_dst_values_dict[x] if x in convert_dst_values_dict.keys() else x)
df['Dst'] = df['Dst'].astype(int)
df = df[df['Dst'] != 0]

In [13]:
df['Dat'] = pd.to_datetime(df['Dat'])

In [14]:
df['Twt'] = df['Twt'].str.replace('\.$', '', regex=True)
df['Twt'] = df['Twt'].str.replace('^0', '', regex=True)
df = df[~df['Twt'].isin(['', '557', '5½', '7'])]
convert_twt_values_dict = {'056':'56', '059':'59', '071':'71', '600':'60', '77.7':'77'}
df['Twt'] = df['Twt'].apply(lambda x: convert_twt_values_dict[x] if x in convert_twt_values_dict.keys() else x)
df['Twt'] = df['Twt'].apply(lambda x: float(x[:-1]) if '½' in x else float(x))

In [15]:
df['Gen'] = np.where(df['Gen'] == 'M', 1 ,0)

In [16]:
df['Wlp'] = pd.to_datetime(df['Wlp']) #subtract from date of race to create age at time of race

In [17]:
df = df[df['Hgd'] != '.']

In [18]:
df['Age'] = df['Dat'] - df['Wlp']
df.drop(columns=['Dat', 'Wlp'], inplace=True)
df['Age'] = df['Age'].dt.days

### EDA

### Basic Modeling

In [19]:
X = pd.get_dummies(df, drop_first=True)
X = X.drop(columns='Fin')
y = df[['Fin']]

In [20]:
X.shape

(488505, 8447)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)

In [22]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

MemoryError: Unable to allocate 23.1 GiB for an array with shape (8447, 366378) and data type float64

In [ ]:
lr.score()